In [1]:
import pandas as pd
import numpy as np
import time
from sqlalchemy import create_engine
!pip install pymysql
import pymysql
from collections import Counter
import math 
from datetime import datetime 
import warnings


In [4]:
engine = create_engine("mysql+pymysql://user345:Password1@@162.243.235.211/db345")
challenge = pd.read_sql_query("SELECT * FROM db345.challenge", engine)
challenge.to_pickle("a_file.pkl")
#challenge = pd.read_pickle("a_file.pkl")

In [5]:
engine = create_engine("mysql+pymysql://user345:Password1@@162.243.235.211/db345")
winers = pd.read_sql_query("SELECT * FROM db345.winner", engine)
winers.to_pickle("a_file_winers.pkl")
#winers = pd.read_pickle("a_file_winers.pkl")
engine = create_engine("mysql+pymysql://user345:Password1@@162.243.235.211/db345")
handle = pd.read_sql_query("SELECT * FROM db345.relation_c_r", engine)
handle.to_pickle("a_file_handle.pkl")
#handle = pd.read_pickle("a_file_handle.pkl")

In [6]:
challenge = pd.read_pickle("a_file.pkl")
winers = pd.read_pickle("a_file_winers.pkl")
handle = pd.read_pickle("a_file_handle.pkl")

In [7]:
import os

my_path = 'C:/not windows/bax/RAZIEH'
# challenge = pd.read_csv(os.path.join(my_path,'challenge.csv'))   
# winers = pd.read_csv(os.path.join(my_path,'winers.csv'))   
# handle = pd.read_csv(os.path.join(my_path,'handle.csv'))   
df_tech = pd.read_csv(os.path.join(my_path,'Technologies2.csv'))   
df_belt = pd.read_csv(os.path.join(my_path,'WorkersBelt.csv')) 
comments = pd.read_csv(os.path.join(my_path,'merged_phase_new.csv')) 
comments=comments[comments["totalPrize"]>0] 
readability = pd.read_csv(os.path.join(my_path,'readability.csv')) 

In [6]:
my_path = 'C:/not windows/bax/RAZIEH/challenge.pkl'
 

In [8]:
filter=pd.isna((readability.challengeId[:]))
sett_1=( readability[filter].index-1).tolist()
sett0=( readability[filter].index).tolist()
sett1=( readability[filter].index+1).tolist()
sett_merge=sett0 + sett1 + sett_1
readability.loc[sett_merge,:]=0
#print(readability.loc[sett_merge]) # readability.drop(sett_merge,inplace=True)
readability['challengeId']=readability['challengeId'].astype(int)

In [9]:
ID_set=(comments['challengeId'])# print(type(readability.loc[1,'challengeId']))
comments.loc[:,'dale_chall_readability_score']=0 
comments['flesch_reading_ease']=0
for ID_i in ID_set: 
    #print(ID_i,comments.loc[comments['challengeId']==ID_i,['dale_chall_readability_score','flesch_reading_ease']])
    #print(ID_i,readability.loc[readability['challengeId']==ID_i,['dale_chall_readability_score','flesch_reading_ease']])
    comments.loc[comments['challengeId']==ID_i,['dale_chall_readability_score','flesch_reading_ease']] =readability.loc[readability['challengeId']==ID_i,['dale_chall_readability_score','flesch_reading_ease']]
    #print(comments.loc[comments['challengeId']==ID_i,['dale_chall_readability_score','flesch_reading_ease']])

In [10]:
columns = df_tech.columns.tolist()

In [11]:
df_techs = pd.melt(df_tech, id_vars=columns[:1], value_vars= columns[1:])
df_techs = df_techs.drop(['variable'], axis=1) #print(df_techs)
df_techs=df_techs [( (df_techs[['value']]!=0 ) & ( df_techs[['value']]!= '0') ).all(axis=1) ]

In [12]:

handle['ch_type']=np.nan
handle['totalPrize']=np.nan  # jupiter version addition
handle['Duration']=np.nan    # jupiter version addition

In [14]:
handle['placement']=0
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)    
for i in handle.index:
######addition of the ch_type, totalPrize, and Duration columns to the handle dataframe
    ChId_i=handle['challengeId'][i] # ChId_i=challenge id for the ith row in the handle matrix
    row_ChId_i=challenge['challengeId']==ChId_i  # the row in the challenge matrix which represent the ChId_i
    type_ChId_i=(challenge.loc[row_ChId_i,'challengeType']).values[0]
    totalPrize=challenge.loc[row_ChId_i,'totalPrize'].values[0]
    duration_ChId_i=(challenge.loc[row_ChId_i,'submissionEndDate']-challenge.loc[row_ChId_i,'registrationStartDate'])
    
    #cnt11=comments[comments['challengeId']==ChId_i]    wrd_cnt= len((cnt11.loc[:,'detailedRequirements'].to_string()))    
    
    handle.loc[[i],['ch_type','totalPrize','Duration']]=[type_ChId_i,totalPrize,duration_ChId_i]
######addition of the placement column to the handle dataframe
    handle_i=handle.loc[i,['challengeId','handle']]
    ros_winer_i=(winers["challengeId"] == handle_i[0]) & (winers["handle"] == handle_i[1])
    non_0_ros_winer_i=ros_winer_i.any()
    if non_0_ros_winer_i==1:
       handle.loc[[i],['placement']]=winers.loc[ros_winer_i,'placement'].values[0]

In [15]:
handle.to_pickle("handle.pkl")
#handle = pd.read_pickle("handle.pkl")

In [16]:
def worldcount(X):
    return len(X.split())

In [17]:
comments['detailedRequirements'] = comments['detailedRequirements'].astype(str)
comments["world_count_revised"] = comments['detailedRequirements'].apply(worldcount)
comments["challenge_difficulty"]= comments["world_count_revised"]/(comments["days"]+1)
comments[["challengeId","totalPrize","world_count_revised","days","numTech","challenge_difficulty"]] 


,challengeId,totalPrize,world_count_revised,days,numTech,challenge_difficulty
0,30033143,2100,262,3,1,65.500000
1,30033144,1800,279,5,1,46.500000
2,30033182,1950,81,8,1,9.000000
3,30033189,2100,226,5,2,37.666667
4,30033190,300,232,2,2,77.333333
...,...,...,...,...,...,...
4426,30048878,40,48,30,1,1.548387
4427,30048879,40,48,30,1,1.548387
4428,30048880,75,48,30,1,1.548387
4429,30048881,20,48,30,1,1.548387


In [18]:
final = handle.merge(df_techs , left_on='challengeId', right_on='challengeId') #print(final)
final = final.drop_duplicates() #print(final)
final = final[final['value'] != "0" ]

In [19]:
#data cleaning
wrk_set=set(final['handle'])
final_clean=final.copy()
TrshHld=10/2     #tune2
cnt=0
for wrk_i in wrk_set:
    ros_wrk_i=final_clean['handle']==wrk_i
    sbst_wrk_i=final_clean[ros_wrk_i]  # corresponding subset of final dataframe for the ith worker,wrk_i
    sbst_wrk_i2=sbst_wrk_i.copy()
    sbst_wrk_i2.drop_duplicates(subset=["challengeId"],inplace = True) #.drop_duplicates(subset=['A', 'B'], keep='last')
    submit_wrk_i=sbst_wrk_i2['submissionDate']=='0000-00-00 00:00:00'  
    zero_sbmit_wrk_i=submit_wrk_i.all()
    regis_wrk_i=sbst_wrk_i2['registrationDate']  #  
    num_regis_wrk_i=len(regis_wrk_i)
    if num_regis_wrk_i<TrshHld or zero_sbmit_wrk_i==1:
      c3=ros_wrk_i==0
      final_clean=final_clean[c3]     #print(final_clean)    #time.sleep(5.5) 
wrk_set2=set(final_clean['handle'])
ros_final_clean_sub=final_clean['submissionDate']!= '0000-00-00 00:00:00' 
final_clean_sub=final_clean[ros_final_clean_sub]

In [20]:
final_clean_sub.to_pickle("final_clean_sub.pkl")
#final_clean_sub = pd.read_pickle("final_clean_sub.pkl")

In [21]:
# Resiliance and difficulty/price for each worker
df_Res_D2P=pd.DataFrame(np.zeros((len(wrk_set2), 2 )),index=wrk_set2, columns=["Resiliance","Diff2PriceTresh"])
wrk_set2=set(final_clean['handle']) 

for wrk_i in wrk_set2:
    #print(wrk_i)
    ChIds_wrk_i=final_clean.loc[final_clean['handle']==wrk_i,"challengeId"]
    ChDiffs_wrk_i=comments.loc[ comments["challengeId"].isin(ChIds_wrk_i),["challenge_difficulty"] ] 
    prices_wrk_i=comments.loc[ comments["challengeId"].isin(ChIds_wrk_i),["totalPrize"] ] 
    Res_wrk_i=sum(ChDiffs_wrk_i["challenge_difficulty"])/len(ChDiffs_wrk_i)
    D2P_wrk_i=sum(ChDiffs_wrk_i["challenge_difficulty"]/prices_wrk_i["totalPrize"])/len(ChDiffs_wrk_i)
    df_Res_D2P.loc[wrk_i,["Resiliance","Diff2PriceTresh"]]=[Res_wrk_i,D2P_wrk_i]
#print(df_Res_D2P)  

In [22]:
import datetime
current_date = challenge['registrationStartDate'].max()
period_duration = datetime.timedelta(20*1)  #tune2
period_strt_date = current_date - period_duration
period_duration

datetime.timedelta(days=20)

In [23]:
ros_final_clean_win=final_clean['placement']!= 0 
final_clean_win=final_clean[ros_final_clean_win]

In [24]:
# function for extraction of of reliaboty and trustability of a worker in a period
def reliability_trustability_i_t( sbst_wrk_i_t):
    REG_no=sbst_wrk_i_t.shape[0]
    
    ros_sub=sbst_wrk_i_t['submissionDate']!='0000-00-00 00:00:00'  
    SUB_no=sbst_wrk_i_t[ros_sub].shape[0]
    
    ros_win=sbst_wrk_i_t['placement']!=0  
    WIN_no=sbst_wrk_i_t[ros_win].shape[0]
    
    reliability_wrki=SUB_no/REG_no
    trustability_wrki=WIN_no/REG_no 
    return reliability_wrki, trustability_wrki  


In [25]:
# extraction of dynamic reliaboity and trustability of the workers as a function of time
index = pd.MultiIndex.from_product([[], []] ,names=['workers', ' performance'])
workers_reli_trust = pd.DataFrame( index=index)
wrk_set2=set(final_clean['handle'])

for wrk_i in wrk_set2:
    #print(wrk_i)
    ros_wrk_i=final_clean['handle']==wrk_i
    sbst_wrk_i=final_clean[ros_wrk_i]
    sbst_wrk_i2=sbst_wrk_i.copy()
    sbst_wrk_i2=sbst_wrk_i2.sort_values(by=['registrationDate'])
    sbst_wrk_i2.drop_duplicates(subset=["challengeId"],inplace = True)  

    idx = pd.IndexSlice 
    index = pd.MultiIndex.from_product([[wrk_i], ['reliability', 'trustability']])
    reli_trust=pd.DataFrame( index=index)   # creating an empty dataframe
    PI=15           # PI=Period of interet in the worker
    iteration=sbst_wrk_i2.shape[0]-(PI-1)
    if (iteration>=1):
      for i in range(iteration):
        sbst_wrk_i2_t=sbst_wrk_i2.iloc[i:i+PI,:]
        #print(sbst_wrk_i2_t[['submissionDate','placement']])      time.sleep(5)
        reliability_wrki_t, trustability_wrki_t=reliability_trustability_i_t(sbst_wrk_i2_t)
        reli_trust.loc[idx[wrk_i,['reliability','trustability']],idx[i]]=[reliability_wrki_t,trustability_wrki_t]
    else:
      sbst_wrk_i2_t=sbst_wrk_i2
      reliability_wrki_t, trustability_wrki_t=reliability_trustability_i_t(sbst_wrk_i2_t)
      reli_trust.loc[idx[wrk_i,['reliability','trustability']],idx[0]]=[reliability_wrki_t,trustability_wrki_t]
    workers_reli_trust=workers_reli_trust.append(reli_trust) 


In [26]:
# Global dataframe for tech-type

tech_set=set(final_clean['value'])

# hierarchical empty indices and columns
index = pd.MultiIndex.from_product([[], []] ,names=['technologies', ' ChlTypes'])
columns = ['type freq registered']
df_Tech_Type = pd.DataFrame( index=index, columns=columns)


for tech_j in tech_set:
          
    ros_tech_j=final_clean['value']==tech_j
    sbst_tech_j=final_clean[ros_tech_j]   
    ChlTypes_tech_j=sbst_tech_j['ch_type']

    ###extracting frequency for the registeration criterion
    freq_nm_ChlTypes_tech_j=Counter(ChlTypes_tech_j) 
    name_ChlTypes_tech_j=freq_nm_ChlTypes_tech_j.keys() 
    freq_ChlTypes_tech_j=freq_nm_ChlTypes_tech_j.values()

    ###extracting frequency for the submition criterion   
    ros_tech_j_sub=sbst_tech_j['submissionDate']!= '0000-00-00 00:00:00'
    ChlTypes_tech_j_sub=ChlTypes_tech_j[ros_tech_j_sub]
    freq_nm_ChlTypes_tech_j_sub=Counter(ChlTypes_tech_j_sub) 
    name_freq_nm_ChlTypes_tech_j_sub=freq_nm_ChlTypes_tech_j_sub.keys() 
    freq_freq_nm_ChlTypes_tech_j_sub=freq_nm_ChlTypes_tech_j_sub.values()

    ###extracting frequency for the winning criterion   
    ros_tech_j_win=sbst_tech_j['placement']!=0
    ChlTypes_tech_j_win=ChlTypes_tech_j[ros_tech_j_win]
    freq_nm_ChlTypes_tech_j_win=Counter(ChlTypes_tech_j_win) 
    name_freq_nm_ChlTypes_tech_j_win=freq_nm_ChlTypes_tech_j_win.keys() 
    freq_freq_nm_ChlTypes_tech_j_win=freq_nm_ChlTypes_tech_j_win.values()

    ### addition of the type freq registered column
    tt=list(name_ChlTypes_tech_j)
    index = pd.MultiIndex.from_product([ [tech_j], tt],names=[ 'technologies', ' ChlTypes'])
    df_tech_j_Types = pd.DataFrame(freq_ChlTypes_tech_j, index=index, columns=columns)
    ### addition of the submition column
    df_tech_j_Types['type freq submited'] = [0]*len(freq_ChlTypes_tech_j)
    idx = pd.IndexSlice
    df_tech_j_Types.loc[ idx[[tech_j],list(name_freq_nm_ChlTypes_tech_j_sub)], idx['type freq submited'] ]=list(freq_freq_nm_ChlTypes_tech_j_sub)
    df_tech_j_Types.loc[ idx[[tech_j],list(name_freq_nm_ChlTypes_tech_j_win)], idx['type freq win'] ]=list(freq_freq_nm_ChlTypes_tech_j_win)#
    fre_reg_sub_win_tech_j=df_tech_j_Types.sum(axis = 0)
    df_tech_j_Types['tech freq registered'] = [ fre_reg_sub_win_tech_j[0] ]*len(freq_ChlTypes_tech_j)
    df_tech_j_Types['tech freq submited'] = [ fre_reg_sub_win_tech_j[1] ]*len(freq_ChlTypes_tech_j)
    df_tech_j_Types['tech freq win'] = [ fre_reg_sub_win_tech_j[2] ]*len(freq_ChlTypes_tech_j)#
    #########
    df_Tech_Type=df_Tech_Type.append(df_tech_j_Types) 


In [27]:
idx = pd.IndexSlice

In [28]:
def evenOdd2( RoS, final_clean_REGoSUB ):     
    wrk_set=set(final_clean_REGoSUB['handle'])
    ### creating 3 empty dataframes for the demonstration of workers correspondence to technologies, types, and budget_duration
    type_list=set(challenge['challengeType'])
    df_wrk2type=pd.DataFrame(np.zeros((len(wrk_set), len(type_list))),index=wrk_set, columns=type_list)
    tech_list=df_tech.columns.tolist()
    df_wrk2tek=pd.DataFrame(np.zeros((len(wrk_set), len(tech_list))),index=wrk_set, columns=tech_list)
    df_wrk2budget_duration=pd.DataFrame(np.zeros((len(wrk_set), 2)),index=wrk_set, columns=['Duration','totalPrize'])
    ### 

    # creating empty dataframes for the mtrices extraction
    index = pd.MultiIndex.from_product([[], [], []],names=['workers', 'technologies', ' tech types'])
    #columns = ['type freq', 'Tot_type_fre','ratio_k', 'ratio_j', 'ratio_i']
    health_data = pd.DataFrame( index=index, columns=columns)

    for wrk_i in wrk_set:
        #sbst_wrk_i=final_clean[ final_clean['handle']==wrk_i ]  # corresponding subset of final dataframe for the ith worker,wrk_i
        sbst_wrk_i=final_clean_REGoSUB[ final_clean_REGoSUB['handle']==wrk_i ]  # corresponding subset of final dataframe for the ith worker,wrk_i
        techs_wrk_i=sbst_wrk_i['value']  # corresponding technologies for the ith worker,wrk_i
        type_wrk_i=sbst_wrk_i[['challengeId','ch_type','totalPrize', 'Duration']]  # corresponding challengetype for the ith worker,wrk_i
        ### worker to type and budget_duration correspondence,df_wrk2type ,df_wrk2budget_duration
        type_wrk_i.drop_duplicates(subset=["challengeId"],inplace = True)

        #type_wrk_i=type_wrk_i.head()

        freq_nm_type_wrk_i=Counter(type_wrk_i['ch_type']) 
        name_type_wrk_i=freq_nm_type_wrk_i.keys() 
        freq_type_wrk_i=freq_nm_type_wrk_i.values()

        df_wrk2type.loc[wrk_i,name_type_wrk_i]=list(freq_type_wrk_i) 
        mean_budget_duration= type_wrk_i[['Duration','totalPrize']].sum(axis = 0)/sum(freq_type_wrk_i) 
        #print(type_wrk_i[['Duration','totalPrize']])
        #print(mean_budget_duration)
        #print(freq_type_wrk_i)
        #time.sleep(5)
        df_wrk2budget_duration.loc[wrk_i,['Duration','totalPrize']]=mean_budget_duration
        ###
        ### worker 2 tech correspondence,df_wrk2tek
        freq_nm_techs_wrk_i=Counter(techs_wrk_i) 
        name_techs_wrk_i=freq_nm_techs_wrk_i.keys() 
        freq_techs_wrk_i=freq_nm_techs_wrk_i.values()
        df_wrk2tek.loc[wrk_i,name_techs_wrk_i]=list(freq_techs_wrk_i) 
        ### 
        ratio_wrk_i_techs_2_all_techs=len(name_techs_wrk_i)/len(tech_set)                          ### diversity_i, diversity for the ith worker
        lst_name_techs_wrk_i=list(name_techs_wrk_i)
        num_techs_wrk_i=len(lst_name_techs_wrk_i)
        VEC_slfprt_wrk_i_tech_jS=[]
        for j in range(num_techs_wrk_i):
            tech_j=lst_name_techs_wrk_i[j]   
            ###### frequency extraction for ch_typeS in tech_j for wrk_i
            sbst_tech_j=sbst_wrk_i[ sbst_wrk_i['value']==tech_j ]   
            ChlType_tech_j=sbst_tech_j['ch_type']   
            
            freq_nm_ChlType_tech_j=Counter(ChlType_tech_j) 
            freq_ChlTypes_tech_j=freq_nm_ChlType_tech_j.values() # frequency of the challenge types which the ith worker has been faced with his jth technology      
            name_ChlTypes_tech_j=freq_nm_ChlType_tech_j.keys() # name of the challenge types which the ith worker has been faced with his jth technology
            
            ######## ###diversity_j, diversity of the jth technology for the ith worker 
            idx = pd.IndexSlice 
            Tot_type_fre2= df_Tech_Type.loc[ idx[tech_j, : ], idx[RoS] ]      #####@@@@@@@@@
            name_global_ChlTypes_tech_j=Tot_type_fre2.index.get_level_values(1)  
            ratio_wrki_techj_Chals_2_all_techj_Chals=len(name_ChlTypes_tech_j)/len(name_global_ChlTypes_tech_j) ###diversity_j, diversity of the jth technology for the ith worker      
            ########
            ##### local and global participation in form of  type
            tt=list(name_ChlTypes_tech_j)

            Tot_type_fre= df_Tech_Type.loc[ idx[tech_j, tt ], idx[RoS] ].values  #####@@@@@@@@@
            VEC_glbprt_wrk_i_tech_j_type_kS = [a / b for a, b in zip(freq_ChlTypes_tech_j, Tot_type_fre)]  ### ratio_k: participation of the ith worker in challenges related to the jth technology

            fre_techj_wrki=sum(freq_ChlTypes_tech_j) #  for i in handle.index:
            VEC_slfprt_wrk_i_tech_j_type_kS = [freq_nm_ChlType_tech_j[k]/fre_techj_wrki for k in name_ChlTypes_tech_j] #@@@@@@@@ chek shavad k shmarande hast na meghdar
            ##### local and global participation in form of technology 
            fre_subset_techj_glb=sum(Tot_type_fre)
            VEC_glbprt_wrk_i_tech_jS=fre_techj_wrki/fre_subset_techj_glb

            Nrow_ChlTypes_wrk_i_tech_j=len(VEC_slfprt_wrk_i_tech_j_type_kS)
            VEC_slfprt_wrk_i_tech_jS0=[ 0 ]* (Nrow_ChlTypes_wrk_i_tech_j-1)
            VEC_slfprt_wrk_i_tech_jS0.insert(0, fre_techj_wrki)
            
            VEC_slfprt_wrk_i_tech_jS.append(fre_techj_wrki) # to be devided by the sum of all fre_techj_wrki, sum(VEC_slfprt_wrk_i_tech_jS)
            

            ratio_wrki_techj_Chals_2_all_techj_Chals2=[0] * Nrow_ChlTypes_wrk_i_tech_j    
            ratio_wrki_techj_Chals_2_all_techj_Chals2[0]=ratio_wrki_techj_Chals_2_all_techj_Chals  #### diversityj

            index = pd.MultiIndex.from_product([[wrk_i], [tech_j], tt],names=['workers', 'technologies', ' tech types'])
            #d={'type freq': freq_ChlTypes_tech_j, 'Tot_type_fre': Tot_type_fre, 'ratio_k': ratio_wrk_i_tech_j_type_K , 'ratio_j': ratio_wrk_i_tech_j , 'ratio_i': [np.nan] * Nrow_ChlTypes_wrk_i_tech_j,
            # 'diversity_i': [np.nan] * Nrow_ChlTypes_wrk_i_tech_j, 'diversity_j': ratio_wrki_techj_Chals_2_all_techj_Chals2   } 
            d={'self participation task type': VEC_slfprt_wrk_i_tech_j_type_kS , 'self participation technology type': VEC_slfprt_wrk_i_tech_jS0 ,\
            'global participation task type': VEC_glbprt_wrk_i_tech_j_type_kS , 'global participation technology type': 0,\
            'diversity_j': ratio_wrki_techj_Chals_2_all_techj_Chals2 , 'diversity_i': [0] * Nrow_ChlTypes_wrk_i_tech_j}

            df_wrk_i_tech_j = pd.DataFrame(d, index=index)
            df_wrk_i_tech_j.loc[ idx[wrk_i,tech_j,df_wrk_i_tech_j.index.get_level_values(2)[0]], idx['global participation technology type'] ]=VEC_glbprt_wrk_i_tech_jS
            health_data=health_data.append(df_wrk_i_tech_j)

        health_data.loc[ idx[wrk_i,:,:], idx['self participation technology type'] ]=\
        health_data.loc[ idx[wrk_i,:,:], idx['self participation technology type'] ]/sum(VEC_slfprt_wrk_i_tech_jS)

        #print(health_data.loc[ idx[wrk_i,:,:], idx['diversity_i'] ].index.get_level_values)
        health_data.loc[ idx[wrk_i,health_data.loc[ idx[wrk_i,:,:], idx['diversity_i'] ].index.get_level_values(1)[0],health_data.loc[ idx[wrk_i,:,:],\
                                                                    idx['diversity_i'] ].index.get_level_values(2)[0]], idx['diversity_i'] ]=ratio_wrk_i_techs_2_all_techs
        #print(health_data.loc[ idx[wrk_i,:,:], idx['diversity_i'] ])
    return health_data, df_wrk2budget_duration, df_wrk2type, df_wrk2tek
            ###########


In [29]:
health_data_REG, df_wrk2budget_duration_REG, df_wrk2type_REG, df_wrk2tek_REG=health_data2=evenOdd2( 'type freq registered',final_clean)  
#health_data_SUB, df_wrk2budget_duration_SUB, df_wrk2type_SUB, df_wrk2tek_SUB=health_data2=evenOdd2( 'type freq submited', final_clean_sub) 


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2886: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


In [30]:
#### network    
# creating 2 DFs for correspondence of the workers to the all and only ongoing tasks ,df_wrks2ChIDs, df_wrks2ChIDs_ongoing
wrk_set2=set(final_clean['handle'])
ChId_set2=set(final_clean['challengeId'])

###### ongoing
ros_ongoing=(challenge['registrationStartDate']<=current_date) & (challenge['registrationStartDate']>=period_strt_date) & (challenge['registrationStartDate']<=current_date)
ChId_set2_ongoing=challenge[ros_ongoing]['challengeId']
ChId_set2_ongoing_clean=ChId_set2_ongoing.loc[(ChId_set2_ongoing.isin(final_clean['challengeId'])) ]
######
                # zeros filled df_wrks2ChIDs, df_wrks2ChIDs_com, df_wrks2ChIDs_ongoing
df_wrks2ChIDs = pd.DataFrame(np.zeros((len(wrk_set2), len(ChId_set2))), index=wrk_set2, columns=ChId_set2) 
###### ongoing
df_wrks2ChIDs_ongoing = pd.DataFrame(np.zeros((len(wrk_set2), len(ChId_set2_ongoing_clean))), index=wrk_set2, columns=ChId_set2_ongoing_clean)
df_wrks2recomChIDs_ongoing = pd.DataFrame(np.zeros((len(wrk_set2), len(ChId_set2_ongoing_clean))), index=wrk_set2, columns=ChId_set2_ongoing_clean)
df_active_wrks2recomChIDs_ongoing_ = pd.DataFrame(np.zeros((len(wrk_set2), len(ChId_set2_ongoing_clean))), index=wrk_set2, columns=ChId_set2_ongoing_clean)
###### ongoing
                #  filling the df_wrks2ChIDs, df_wrks2ChIDs_ongoing
for wrk_i in wrk_set2:
  ros_wrk_i=final_clean['handle']==wrk_i
  ChIds_wrk_i=set(final_clean['challengeId'][ros_wrk_i]) 
  df_wrks2ChIDs.loc[wrk_i,ChIds_wrk_i]=1 
  ###### ongoing
  ChIds_wrk_i_ongoing=ChId_set2_ongoing.loc[(ChId_set2_ongoing.isin(ChIds_wrk_i)) ]
  df_wrks2ChIDs_ongoing.loc[wrk_i,ChIds_wrk_i_ongoing]=1  
  ###### ongoing

                # zero filled df_wrks2wrks to show common tasks between workers ,df_wrks2wrks
df_wrks2wrks = pd.DataFrame(np.zeros((len(wrk_set2), len(wrk_set2))), index=wrk_set2, columns=wrk_set2) 
                #  filling the df_wrks2wrks
                    #  there is an external for-loop layer with i or wrk_i as counter. The external layer has 2 internal layer with j and p as
                    #  cuonters for each iteration, wrk_i, in the external layer the similar workers are identified and the number of shared
                    #  task with each worker is stored in the ith row of the df_wrks2wrks DF
wrk_list2=list(wrk_set2)
cnt=0
typ_fav_trshol=0.2 *1                                                     #tuning parameter :minimum treshhold for favourity of a type for a worker
tek_fav_trshol=0.3 *1                                                        #tuning parameter :minimum treshhold for favourity of a technology for a worker
#print(len(wrk_set2))
for i in range(len(wrk_set2)):
  #print(i)

  hd=0
  wrk_i=wrk_list2[i]
  w_i=df_wrks2ChIDs.loc[wrk_i,:]
  ###### ongoing
  #w_i_com=df_wrks2ChIDs_com.loc[wrk_i,:] 
  w_i_ongoing=df_wrks2ChIDs_ongoing.loc[wrk_i,:] 
  ###### ongoing
  ######tools for filtering the workers for recommendation  submissionDate registrationDate preferences
  ros_wrk_i=final_clean['handle']==wrk_i
  submissionDate_revised=pd.to_datetime(final_clean['submissionDate'],errors ='coerce')-datetime.timedelta(30)  # revising the column because it is not from a datetime format df['Time'] = df['Time'].astype('datetime64')
  last_sumb_wrk_i=submissionDate_revised[ros_wrk_i].max()
  last_regis_wrk_i=(final_clean['registrationDate'][ros_wrk_i]).max() 
  wrki_regis_activity=last_regis_wrk_i>(current_date-datetime.timedelta(30))    # tuning parameter :period for at least 1   registeration  to be qualified to receive a recommendation
  wrki_sumb_activity=last_sumb_wrk_i>(current_date-datetime.timedelta(30*2))    # tuning parameter :period for at least 1   sumission      to be qualified to receive a recommendation
  fre_all_ChTyps_wrk_i=df_wrk2type_REG.loc[wrk_i,:].sum()  # the frequency of the proposed task,c, will be extracted in the last for loop
  fre_all_TekTyps_wrk_i=df_wrk2tek_REG.loc[wrk_i,:].sum()  # the frequency of the proposed task,c, will be extracted in the last for loop
  #######
                    # Start of the first internal for loop
  for j in range(i+1, len(wrk_set2)):  
    if i == len(wrk_set2)-1:   ## the last row does not need to be filled since it has already been filled because of the symmetry condition
        break
    wrk_j=wrk_list2[j]
    w_j=df_wrks2ChIDs.loc[wrk_j,:]

    vec_0_1_2=w_i+w_j  ## ind_onlyj=(w_i_com==0) & (w_j_com==1) df_wrks2ChIDs_com.columns[ind_onlyj]
    shrd_tsks_ind=vec_0_1_2==2
    shrd_tsks=vec_0_1_2[shrd_tsks_ind]

    similarity_ij=len(shrd_tsks)
    df_wrks2wrks.loc[wrk_i,wrk_j]=similarity_ij
    df_wrks2wrks.loc[wrk_j,wrk_i]=similarity_ij # the symmetry propery of the df_wrks2wrks DF
                    # End of the first internal for loop
                #  continuation of the external for loop for extracting the similar workers among all workers for wrk_i as wrk_i_pairs_sort_ind;
                #  and preferences of the wrk_i
  wrk_i_pairs=(df_wrks2wrks.loc[wrk_i,:]).copy()  
  wrk_i_pairs_sort=wrk_i_pairs.sort_values(ascending = False)
  wrk_i_pairs_sort=wrk_i_pairs_sort.loc[wrk_i_pairs_sort>0]
  wrk_i_pairs_sort_ind=wrk_i_pairs_sort.index

  Resiliance_wrk_i=df_Res_D2P.loc[wrk_i,"Resiliance"] #  Cst_min_wrk_i=(final_clean.loc[final_clean['challengeId'].isin(w_i[w_i==1].index),'totalPrize']).min()
  Time_min_wrk_i=(final_clean.loc[final_clean['challengeId'].isin(w_i[w_i==1].index),'Duration']).min()
  ChTyps_wrk_i=(final_clean.loc[final_clean['challengeId'].isin(w_i[w_i==1].index),'ch_type']).unique() #
  Techs_wrk_i=(final_clean.loc[final_clean['challengeId'].isin(w_i[w_i==1].index),'value']).unique()
                     # Start of the 2nd internal for loop over the workers similar to wrk_i
  for p in wrk_i_pairs_sort_ind:
      #print(p , wrk_i)
  ###### ongoing
      w_p_ongoing=df_wrks2ChIDs_ongoing.loc[p,:]  
      ChIdS_onlyp=(w_p_ongoing[w_p_ongoing==1].index)[(w_p_ongoing[w_p_ongoing==1].index).isin(w_i[w_i==0].index)]   
  ###### ongoing

                      # Start of the 2nd_prime internal for loop over the challenge IDs for worker p and not shared with worker i
                      # properties correspond to preferences of a worker is derived and if there was a level of similarities between them the
                      # challengeID c will be stored to be recommended to the worker i
      recom_to_wrk_i=[] 
      for c in ChIdS_onlyp:
        wrk_p_Ch_c_difficulty=comments.loc[comments["challengeId"]==c,"challenge_difficulty"]  # wrk_p_Ch_c_cst=final_clean.loc[final_clean['challengeId']==c]['totalPrize'].unique()[0]
        wrk_p_Ch_c_type=final_clean.loc[final_clean['challengeId']==c]['ch_type'].unique()[0]
        wrk_p_Ch_c_techs=final_clean.loc[final_clean['challengeId']==c]['value'].unique()
        wrk_p_Ch_c_duration=final_clean.loc[final_clean['challengeId']==c]['Duration'].unique()[0]
        ###favourity for type c
        fre_c_ChTyp_wrk_i=df_wrk2type_REG.loc[wrk_i,wrk_p_Ch_c_type]  
        favority_type_c_for_wrk_i=fre_c_ChTyp_wrk_i/fre_all_ChTyps_wrk_i  # poropsed task,c, favourity for the wrk_i
        ###favourity for teck c
        ros_ChID_c=final_clean['challengeId']==c
        number_of_teks_c=len(wrk_p_Ch_c_techs)         #teks_ChID_c=(final_clean['value'][ros_ChID_c]).drop_duplicates()
        fre_c_TekTyp_wrk_i=df_wrk2tek_REG.loc[wrk_i,wrk_p_Ch_c_techs]  
        overall_favority_teks_c_for_wrk_i=(fre_c_TekTyp_wrk_i/fre_all_TekTyps_wrk_i).sum()  # poropsed task,c, favourity for the wrk_i
        #### 
        # 1st condition: if the Difficulty(length/duration) is lower than the worker Resiliance(mean difficulty tolerance)  
        cond1=Resiliance_wrk_i>wrk_p_Ch_c_difficulty
        if (sum(comments["challengeId"]==c)==0):

            continue
        #print(c,Resiliance_wrk_i,wrk_p_Ch_c_difficulty,cond1, sep='\n')
        #print(wrk_p_Ch_c_difficulty)
        #print(sum(comments["challengeId"]==c))

        # 2nd condition: if the TIME DURATION  is higher than the worker minimum received reward                                              
        cond2=Time_min_wrk_i<wrk_p_Ch_c_duration
        # 3rd condition: if the measurement of the task TYPE FAVOURITY for the worker is higher than the minimum treshhold for type favourity                                             
        cond3=favority_type_c_for_wrk_i>= typ_fav_trshol 
        # 4th condition: if the measurement of the overall task TECHNOLOGIES FAVOURITY for the worker is higher than HALF of minimum treshhold for technology favourity
          #multiplied by the number of technologies in the task
        cond4=overall_favority_teks_c_for_wrk_i>=(tek_fav_trshol* number_of_teks_c)/2  
        if (int(cond1)+int(cond2)+int(cond3)>=2) and  (cond4==1): # from the 1st to 3rd conditions, 2 conditions should be met, and the 4th condition should be met.

  ###### ongoing 
           if (wrki_regis_activity and wrki_sumb_activity):
             hd=1
             df_active_wrks2recomChIDs_ongoing_.loc[wrk_i,c]=1 
             df_wrks2recomChIDs_ongoing.loc[wrk_i,c]=1          
           else: 
             df_wrks2recomChIDs_ongoing.loc[wrk_i,c]=1
  cnt=cnt+hd
  ###### ongoing 
#print(cnt) # numbet of workers who have met the registration and sumbmition condition.
recom_num=df_active_wrks2recomChIDs_ongoing_.sum(axis=1) #number of recommneded task to all workers
#print(recom_num[recom_num>0]) #number of recommneded task to workers with T least on recommendation
#print(df_active_wrks2recomChIDs_ongoing_.sum(axis=0)) 
#        print(wrk_i,Resiliance_wrk_i,wrk_p_Ch_c_difficulty,cond1, sep='\n')

In [31]:
df_active_wrks2recomChIDs_ongoing_.to_pickle("df_active_wrks2recomChIDs_ongoing.pkl")
#df_active_wrks2recomChIDs_ongoing_ = pd.read_pickle("df_active_wrks2recomChIDs_ongoing.pkl")

In [32]:
df_active_wrks2recomChIDs_ongoing_     
df_active_wrks2recomChIDs_ongoing_2 = df_active_wrks2recomChIDs_ongoing_[(df_active_wrks2recomChIDs_ongoing_.T != 0).any()]
df_active_wrks2recomChIDs_ongoing_2 = df_active_wrks2recomChIDs_ongoing_2.loc[:,(df_active_wrks2recomChIDs_ongoing_2 != 0).any()]

In [33]:
Sim=pd.read_pickle("./df_similarity_tasks_pickle.pkl")

In [34]:
#challenge2=challenge.copy()
comments2=comments.copy()

In [35]:
def Six_Sigma_ros(history):
    price=history.loc[:,'totalPrize']
    #comments_ros_01=comments['challengeId'].isin(comments_ros)
    #prices=comments.loc[comments_ros_01,['totalPrize']]
    minn=price.max()
    maxx=price.min()
    mn1=price.mean()
    std1=price.std()
    lower_limit=mn1-1*std1
    upper_limit=mn1+1*std1
    filtered_comments_ros= (price>= lower_limit) & (price<= upper_limit)
    if  (filtered_comments_ros.all()):
        filter_status='no filtering'
#         filtered_comments_ros_IDS=[]
        filtered_comments_ros_IDS=history
    else:
        filter_status='filtered tasks'   
        filtered_comments_ros_IDS=history.loc[ filtered_comments_ros ]
    return filter_status, filtered_comments_ros_IDS

In [36]:
def regression_func2(history, id_i_variables):
    # Step 1: Import packages
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import PolynomialFeatures

    # Step 2a: Provide data challengeId
    ros=comments2["challengeId"].isin(ID_old_W_id_i) #ros=challenge['challengeType']=="First2Finish" 
    #print((ros)) 
    
    if (len(history['world_count_revised'])<=2): #len(comments.loc[ros,['world_count_revised']]) == 0:
        return 'No_similar_task'
    
    x1=history.loc[:, Lst_vari ] #x1=history.loc[:,['numTech','days','world_count_revised']] 
    y1=history.loc[:,'totalPrize'] #(comments2.loc[ros,['totalPrize']]) # y1=(challenge.loc[ros,['numRegistrants']])
    x=np.array(x1.values.tolist())
    y=np.array(y1.values.tolist()) 
    #print(comments2.loc[ros,['numTech','days','world_count_revised','totalPrize']])

    # Step 2b: Transform input data
    deg=1
    x_ = PolynomialFeatures(degree=deg, include_bias=False).fit_transform(x)

    # Step 3: Create a model and fit it
    model = LinearRegression().fit(x_, y)

    # Step 4: Get results
    r_sq = model.score(x_, y)
    intercept, coefficients = model.intercept_, model.coef_

    # Step 5: Predict
    y_pred = model.predict(x_)

    x_id_i = PolynomialFeatures(degree=deg, include_bias=False).fit_transform(id_i_variables)
    y_id_i = model.predict(x_id_i)

    #print(id_i_variables_3,id_i_variables, sep='\n')
    return np.round( y_id_i )

In [37]:
def regression_func2_prime(history, id_i_variables, accepted_vars):
    # Step 1: Import packages
    import numpy as np
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import PolynomialFeatures

    # Step 2a: Provide data challengeId
    
    if (len(history['world_count_revised'])==0):  
        return 'No_similar_task'
    
    if (len(accepted_vars)==0):  
        return 'Non_standard_vars'
    
    x1=history.iloc[:,accepted_vars]  # x1=history.loc[:,['numTech','days','world_count_revised']]  
    y1=history.loc[:,'totalPrize']  
    x=np.array(x1.values.tolist())
    y=np.array(y1.values.tolist()) 

    # Step 2b: Transform input data
    deg=1
    x_ = PolynomialFeatures(degree=deg, include_bias=False).fit_transform(x)

    # Step 3: Create a model and fit it
    model = LinearRegression().fit(x_, y)

    # Step 4: Get results
    r_sq = model.score(x_, y)
    intercept, coefficients = model.intercept_, model.coef_

    # Step 5: Predict
    y_pred = model.predict(x_)

    x_id_i = PolynomialFeatures(degree=deg, include_bias=False).fit_transform(id_i_variables)
    y_id_i = model.predict(x_id_i)

    #print(id_i_variables_3,id_i_variables, sep='\n')
    return np.round( y_id_i )

In [88]:
belt_listS=[['gray ','green ' ,'blue ','yellow ','red'],['gray'],['blue ','green'],['yellow']]
belt_list=[];
for j in range(4): #IDs_recom
    belt_listj=''.join([str(item) for item in belt_listS[j] ])
    belt_list.append(belt_listj)
IDs_recom = df_active_wrks2recomChIDs_ongoing_2.columns.tolist()

index = pd.MultiIndex.from_product([IDs_recom, belt_list])
clmns=["task_type","actual price","filter status","price status","predicted price","upper limit price","lower limit price", "crit_mn", "mean_reli","mean_trst","tot_reli","tot_trst"]
prediction_info6=pd.DataFrame( index=index, columns=clmns)   # creating an empty dataframe 
price_info6=pd.DataFrame( index=index, columns=["price set"])   # creating an empty dataframe 
ID_info6=pd.DataFrame( index=index, columns=["ID set"])   # creating an empty dataframe 

In [90]:
def Six_Sigma_ros_prime(history,var_id):
    list_var=['numTech','days','world_count_revised'] 
    for j_var_id in range(3):
        set_i_r_i=history.iloc[:,1+j_var_id]
        minn=set_i_r_i.max()
        maxx=set_i_r_i.min()
        mn1=set_i_r_i.mean()
        std1=set_i_r_i.std()
        lower_limit=mn1-.5*std1
        upper_limit=mn1+.5*std1
        #print( ( (var_id.iloc[0,j_var_id]< lower_limit) | (var_id.iloc[0,j_var_id]> upper_limit) ) & all( set_i_r_i!=var_id.iloc[0,j_var_id] ) )
        #time.sleep(2)
        if( ( (var_id.iloc[0,j_var_id]< lower_limit) | (var_id.iloc[0,j_var_id]> upper_limit) ) & all( set_i_r_i!=var_id.iloc[0,j_var_id] ) ): ### rrr
            del list_var[j_var_id]         #   var_id_revised0.add(var_id[j_var_id]) bbb
            #print("Six_Sigma_ros_prime",history,var_id,list_var,sep="\n")
            
        return(list_var)

In [45]:
def reli_trust(): #ean_reli, mean_trust, tot_reliability_id_i_belt_j,tot_trustability_id_i_belt_j= reli_trust()
    # setting the initial values of sumation for trust and reliability to zero
        sum_reli=0
        sum_trust=0

        sum_reg=0 
        sum_sub=0    
        sum_win=0
        
        for w_j_id_i in W_id_i:
    #START TRUST AD RELIABILITY EXTRACTIN
            ##### extraction of REG_no, SUB_no, and WIN_no for worker w_j_id_i in worker set W_id_i correspond to new task id_i  
            ros_wrk_i=final_clean['handle']==w_j_id_i
            sbst_wrk_i=final_clean[ros_wrk_i]
            sbst_wrk_i2=sbst_wrk_i.copy()
            sbst_wrk_i2=sbst_wrk_i2.sort_values(by=['registrationDate'])
            sbst_wrk_i2.drop_duplicates(subset=["challengeId"],inplace = True)
            sbst_ID_old_sim_w_j_id_i=sbst_wrk_i2.loc[ (sbst_wrk_i2["challengeId"].isin(ID_old_W_id_i_2))  ,:]
            
            REG_no=sbst_ID_old_sim_w_j_id_i.shape[0] 
            sum_reg=sum_reg+ REG_no
           

            ros_sub=sbst_ID_old_sim_w_j_id_i['submissionDate']!='0000-00-00 00:00:00'  
            sbst_ID_old_sim_w_j_id_i_sub=sbst_ID_old_sim_w_j_id_i[ros_sub]  
            SUB_no=sbst_ID_old_sim_w_j_id_i_sub.shape[0]  
            sum_sub=sum_sub+SUB_no #aaa

            ros_win=sbst_ID_old_sim_w_j_id_i['placement']!=0  
            WIN_no=sbst_ID_old_sim_w_j_id_i[ros_win].shape[0]
            sum_win=sum_win+ WIN_no

            ####
            
            ##### calling and addition the reliability and trust for worker w_j_id_i in worker set W_id_i correspond to new task id_i  
            w_j_id_i_rel_trst=workers_reli_trust.loc[idx[ [w_j_id_i],['reliability','trustability']] ,idx[:]]
            w_j_id_i_rel_trst=w_j_id_i_rel_trst.dropna(axis=1, how='all')

            sum_reli=sum_reli+( w_j_id_i_rel_trst[w_j_id_i_rel_trst.columns[-1]] )[0] 
            sum_trust=sum_trust+( w_j_id_i_rel_trst[w_j_id_i_rel_trst.columns[-1]] )[1]
            ####
            
        ##### extraction of mean_trust and mean_reli  
        if (len(W_id_i)!=0):
            mean_reli=sum_reli/len(W_id_i)
            mean_trust=sum_trust/len(W_id_i)
        else:
            mean_reli=0 
            mean_trust=0
        ##### extraction of tot_reliability_id_i_belt_j and tot_trustability_id_i_belt_j 
        if (sum_reg!=0):
            tot_reliability_id_i_belt_j=sum_sub/sum_reg 
            tot_trustability_id_i_belt_j=sum_win/sum_reg 
        else:
            tot_reliability_id_i_belt_j=0
            tot_trustability_id_i_belt_j=0
        return ( mean_reli, mean_trust, tot_reliability_id_i_belt_j, tot_trustability_id_i_belt_j )
    #END TRUST AD RELIABILITY EXTRACTIN

In [94]:
### new pricing
IDs_recom = df_active_wrks2recomChIDs_ongoing_2.columns.tolist()
module=[1,1,.1] 

comments2=comments.copy()
comments2.loc[:,['numTech']]=round((comments.loc[:,['numTech']])*module[0]) 
comments2.loc[:,['days']]=round((comments.loc[:,['days']])*module[1]) 
comments2.loc[:,['world_count_revised']]=round((comments.loc[:,['world_count_revised']])*module[2])+1 

belt_listS_0=[['gray','green' ,'blue','yellow','red'],['gray'],['blue','green'],['yellow']]
belt_listS=[['gray ','green ' ,'blue ','yellow ','red'],['gray'],['blue ','green'],['yellow']]
idx = pd.IndexSlice 
index = pd.MultiIndex.from_product([[], []])
clmns=["task_type","actual price","filter status","price status","predicted price","upper limit price","lower limit price","crit_mn","mean_reli","mean_trst"]
#prediction_info=pd.DataFrame( index=index, columns=clmns)   # creating an empty dataframe 
cnt2=[]
for j in range(4): 
    ros_belt_listj=(df_belt["Belt"]).isin(belt_listS_0[j])
    W_belt_listj=df_belt.loc[ros_belt_listj,"handle"]
    
        
    for id_i in IDs_recom:
        id_i_real_prize=((comments2.loc[ comments2['challengeId']==id_i,'totalPrize' ]))

        # W_id_i= worker set correspond to workers that the id_i th task has been offered to
        W_id_i=df_active_wrks2recomChIDs_ongoing_2[df_active_wrks2recomChIDs_ongoing_2[id_i]!=0].index 
        ros_W_id_j=(W_id_i).isin(W_belt_listj)
        W_id_i=W_id_i[ros_W_id_j] #W_id_i_belt_listj
               
        # ch_typ_id_i= task type correspond to the id_i th task 
        ch_typ_id_i=comments.loc[comments['challengeId']==id_i,'challengeType'].item()
        #no use now: id_i_registrants= number of resgisterations correspond to the id_i th task    id_i_registrants=comments.loc[comments['challengeId']==id_i,'numRegistrants'].item() 
        # id_i_price= price correspond to the id_i th task 
        id_i_price=comments.loc[ comments['challengeId']==id_i,['totalPrize'] ].values

        # ID_old_W_id_i=challengeIDs correspond to W_id_i set
        ID_old_W_id_i_0=final_clean.loc[ (final_clean["handle"].isin(W_id_i)) & (final_clean["ch_type"]==ch_typ_id_i) ,["challengeId"]]
        ID_old_W_id_i=(ID_old_W_id_i_0['challengeId'])  
        # filteration for the tasks avaialable in the Similarity matrix
        ID_old_W_id_i=(ID_old_W_id_i[ID_old_W_id_i.isin(Sim.columns)]).unique()
        # filteration for the tasks has a similarity higher than >=.8
        ID_old_W_id_i_2=Sim.loc[ id_i, (ID_old_W_id_i) ]
        ID_old_W_id_i_2=(ID_old_W_id_i_2 [Sim.loc[ id_i, ID_old_W_id_i ]>=.4]).index.to_numpy()

        #history_ID_old_W_id_i_2= history of the ID_old_W_id_i_2
        ros=comments2["challengeId"].isin(ID_old_W_id_i_2) #ros=challenge['challengeType']=="First2Finish" 
        history_ID_old_W_id_i_2=comments2.loc[ros,["challengeId",'numTech','days','world_count_revised','totalPrize']]
        #filtered_history_ID_old_W_id_i_2= filteration of history_ID_old_W_id_i_2
        filter_status,filtered_history_ID_old_W_id_i_2=Six_Sigma_ros(history_ID_old_W_id_i_2)    #print(x,y, sep='\n')
               

        # no use now: id_i_variables_01= avarage value for the regidteration number for the similar tasks to id_i th task,ID_old_W_id_i_2.    ID_old_W_id_i_2_ave_rgst=comments.loc[ (comments["challengeId"].isin(ID_old_W_id_i_2)) ,["numRegistrants"]]    id_i_variables_01=((ID_old_W_id_i_2_ave_rgst.mean()).values).reshape(-1,1)
        # id_i_variables= desigh variables correspond to the id_i th task. variable manipulatioins to tune the regression model
        id_i_variables_03=comments.loc[ comments['challengeId']==id_i,['numTech','days','world_count_revised'] ]
        id_i_variables_03.iloc[0,0]=round(id_i_variables_03.iloc[0,0]*module[0])
        id_i_variables_03.iloc[0,1]=round(id_i_variables_03.iloc[0,1]*module[1])
        id_i_variables_03.iloc[0,2]=(round(id_i_variables_03.iloc[0,2]*module[2]))+1

        Lst_vari=Six_Sigma_ros_prime(filtered_history_ID_old_W_id_i_2 ,id_i_variables_03) # price111
        #print("Lst_vari",Lst_vari,sep="\n") 

        id_i_variables_02=id_i_variables_03.loc[:,Lst_vari]
        id_i_variables=( ( id_i_variables_02.values )  ).reshape( -1,len(Lst_vari) )

         #   id_i_variables_final[].append(id_i)
        #Six_Sigma_ros_prime(filtered_history_ID_old_W_id_i_2, id_i_variables):
            
        # price prediction for the primary history
        predict_history_ID_old_W_id_i_2=regression_func2(filtered_history_ID_old_W_id_i_2, id_i_variables)

        #### valid or not  ####
        warnings.simplefilter(action='ignore', category=FutureWarning)
        if (predict_history_ID_old_W_id_i_2!="No_similar_task"):
            if ((predict_history_ID_old_W_id_i_2.astype(int))<0):
                 cnt2.append(id_i)#, print(id_i_variables, filtered_history_ID_old_W_id_i_2, sep="\n"), time.sleep(.020)
        #     print(predict_history_ID_old_W_id_i_2!='No_similar_task')
            price_set=filtered_history_ID_old_W_id_i_2.loc[:,'totalPrize']
            mn2=price_set.mean()
            std2=price_set.std()
            lower_limit2=mn2-1*std2
            upper_limit2=mn2+1*std2
            
                        
            #criteria for price evaluation
            crctns_crit_mn=  abs(predict_history_ID_old_W_id_i_2-mn2 )/ ( mn2 ) 
            crctns_crit_mn=round(crctns_crit_mn[0]*100)

            
            if ( (predict_history_ID_old_W_id_i_2>= lower_limit2) & (predict_history_ID_old_W_id_i_2<= upper_limit2) ):
                price_stat="valid price"   
            else:
                price_stat="invalid price"
                #criteria for price evaluation
                crctns_crit_mn= "invalid price"
        else:
            price_stat="No_similar_task"
            lower_limit2="No_similar_task"
            upper_limit2="No_similar_task"
            
            #criteria for price evaluation
            crctns_crit_mn= "No_similar_task"


    #START TRUST AD RELIABILITY EXTRACTIN
        mean_reli, mean_trust, tot_reliability_id_i_belt_j, tot_trustability_id_i_belt_j= reli_trust()
    #END TRUST AD RELIABILITY EXTRACTIN 
        
    
        belt_listj=''.join([str(item) for item in belt_listS[j] ])
        prediction_info6.loc[idx[id_i, belt_listj],["task_type","actual price","filter status","price status","predicted price","upper limit price","lower limit price", "crit_mn","mean_reli","mean_trst","tot_reli","tot_trst"] ]\
        =[ch_typ_id_i,id_i_real_prize.values,filter_status, price_stat, predict_history_ID_old_W_id_i_2,upper_limit2 ,lower_limit2,  crctns_crit_mn, round(mean_reli,2),round(mean_trust,2), round(tot_reliability_id_i_belt_j,2), round(tot_trustability_id_i_belt_j,2)]

        price_info6.loc[idx[id_i, belt_listj],["price set"] ]= [ [filtered_history_ID_old_W_id_i_2['totalPrize'].tolist(),len(filtered_history_ID_old_W_id_i_2['totalPrize'].tolist()) ] ]      
        ID_info6.loc[idx[id_i, belt_listj],["ID set"] ]= [ [filtered_history_ID_old_W_id_i_2['challengeId'].tolist(),len(filtered_history_ID_old_W_id_i_2['challengeId'].tolist()) ] ]      

   ##### bbb

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in less
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in greater
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in less
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in greater
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in less
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in greater
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in less
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\sit

In [98]:
prediction_info6.head(50)

task_type actual price  \
30048887 gray green blue yellow red  First2Finish        [300]   
         gray                        First2Finish        [300]   
         blue green                  First2Finish        [300]   
         yellow                      First2Finish        [300]   
30048880 gray green blue yellow red  First2Finish         [75]   
         gray                        First2Finish         [75]   
         blue green                  First2Finish         [75]   
         yellow                      First2Finish         [75]   
30048881 gray green blue yellow red  First2Finish         [20]   
         gray                        First2Finish         [20]   
         blue green                  First2Finish         [20]   
         yellow                      First2Finish         [20]   
30048879 gray green blue yellow red  First2Finish         [40]   
         gray                        First2Finish         [40]   
         blue green                  First2Finish         [40]   
         yellow                      First2Finish         [40]   
30048878 gray green blue yellow red  First2Finish         [40]   
         gray                        First2Finish         [40]   
         blue green                  First2Finish         [40]   
         yellow                      First2Finish         [40]   
30048877 gray green blue yellow red  First2Finish         [20]   
         gray                        First2Finish         [20]   
         blue green                  First2Finish         [20]   
         yellow                      First2Finish         [20]   
30048875 gray green blue yellow red  First2Finish         [60]   
         gray                        First2Finish         [60]   
         blue green                  First2Finish         [60]   
         yellow                      First2Finish         [60]   
30048874 gray green blue yellow red  First2Finish         [40]   
         gray                        First2Finish         [40]   
         blue green                  First2Finish         [40]   
         yellow                      First2Finish         [40]   
30048872 gray green blue yellow red  First2Finish         [40]   
         gray                        First2Finish         [40]   
         blue green                  First2Finish         [40]   
         yellow                      First2Finish         [40]   
30048806 gray green blue yellow red  First2Finish        [150]   
         gray                        First2Finish        [150]   
         blue green                  First2Finish        [150]   
         yellow                      First2Finish        [150]   
30048810 gray green blue yellow red  First2Finish        [400]   
         gray                        First2Finish        [400]   
         blue green                  First2Finish        [400]   
         yellow                      First2Finish        [400]   
30048776 gray green blue yellow red  First2Finish        [300]   
         gray                        First2Finish        [300]   
         blue green                  First2Finish        [300]   
         yellow                      First2Finish        [300]   
30048777 gray green blue yellow red  First2Finish        [150]   
         gray                        First2Finish        [150]   

                                      filter status     price status  \
30048887 gray green blue yellow red  filtered tasks      valid price   
         gray                        filtered tasks      valid price   
         blue green                  filtered tasks      valid price   
         yellow                        no filtering  No_similar_task   
30048880 gray green blue yellow red  filtered tasks      valid price   
         gray                        filtered tasks      valid price   
         blue green                  filtered tasks      valid price   
         yellow                      filtered tasks  No_similar_task   
30048881 gray green blue 

In [69]:
id_i_variables_03=comments.loc[ comments['challengeId']==30048675,['numTech','days','world_count_revised'] ]
id_i_variables_03

,numTech,days,world_count_revised
4398,4,30,319


In [70]:
prediction_info6.head(50)
hd=ID_info6.loc[idx[30048675, "blue green"],["ID set"] ][0][0]
hd2=comments.loc[:,"challengeId"].isin(hd)
hd3=comments.loc[hd2,["numTech","days","world_count_revised","totalPrize"]]
hd3
#Six_Sigma_ros_prime(hd3,id_i_variables_03)


TypeError: 'float' object is not subscriptable

In [49]:
cnt2

[30048675, 30048675]

In [71]:
prediction_info6.loc[idx[30048675, :],["task_type","actual price","filter status","price status","predicted price","upper limit price","lower limit price","mean_reli","mean_trst","tot_reli","tot_trst"] ]

task_type actual price  \
30048675 gray green blue yellow red  First2Finish        [300]   
         gray                                 NaN          NaN   
         blue green                           NaN          NaN   
         yellow                               NaN          NaN   

                                      filter status   price status  \
30048675 gray green blue yellow red  filtered tasks  invalid price   
         gray                                   NaN            NaN   
         blue green                             NaN            NaN   
         yellow                                 NaN            NaN   

                                    predicted price upper limit price  \
30048675 gray green blue yellow red        [-120.0]           143.088   
         gray                                   NaN               NaN   
         blue green                             NaN               NaN   
         yellow                                 NaN               NaN   

                                    lower limit price mean_reli mean_trst  \
30048675 gray green blue yellow red           47.4115      0.66      0.66   
         gray                                     NaN       NaN       NaN   
         blue green                               NaN       NaN       NaN   
         yellow                                   NaN       NaN       NaN   

                                    tot_reli tot_trst  
30048675 gray green blue yellow red     0.62     0.62  
         gray                            NaN      NaN  
         blue green                      NaN      NaN  
         yellow                          NaN      NaN

In [56]:
filtered_history_ID_old_W_id_i_2.iloc[270:,1]

Series([], Name: numTech, dtype: int64)

In [57]:
cnt2

[30048675, 30048675]

In [58]:
ID_info6 #
( ID_info6.loc[idx[30048675, "gray"],["ID set"] ] )
# id_i

ID set    [[], 0]
Name: (30048675, gray), dtype: object

In [ ]:
filtered_history_ID_old_W_id_i_2
comments2

In [ ]:
cnt2

In [59]:
(cnt2)
hd=comments2.loc[ comments2["challengeId"].isin(cnt2),["challengeId",'numTech','days','world_count_revised'] ] 
#id_i_variables_02=((comments.loc[ comments['challengeId']==id_i,['numTech','days','world_count_revised'] ]).values).reshape(-1,3)
hd

,challengeId,numTech,days,world_count_revised
4398,30048675,4,30,33.0


In [ ]:
price_info6.loc[idx[30048681, :], : ]

In [ ]:
prediction_info6.loc[idx[30048764, :], : ]
(prediction_info6.loc[idx[30048681, :], : ])


In [ ]:
hh=prediction_info6.loc[idx[:,"yellow" ],["predicted price"]]!="No_similar_task"
prediction_info6.loc[idx[hh,"yellow" ],["predicted price"]]

In [96]:
#hh=prediction_info6.loc[idx[:, "predicted price"]]!="No_similar_task"


ros=prediction_info6.loc[idx[:,: ],"task_type"]!="First2Finish"
prediction_info6.loc[ros].head(50)

task_type actual price  \
30048744 gray green blue yellow red                      Code       [1200]   
         gray                                            Code       [1200]   
         blue green                                      Code       [1200]   
         yellow                                          Code       [1200]   
30048764 gray green blue yellow red                      Code       [1500]   
         gray                                            Code       [1500]   
         blue green                                      Code       [1500]   
         yellow                                          Code       [1500]   
30048758 gray green blue yellow red                      Code       [2100]   
         gray                                            Code       [2100]   
         blue green                                      Code       [2100]   
         yellow                                          Code       [2100]   
30048556 gray green blue yellow red                      Code       [1800]   
         gray                                            Code       [1800]   
         blue green                                      Code       [1800]   
         yellow                                          Code       [1800]   
30048740 gray green blue yellow red      Assembly Competition        [750]   
         gray                            Assembly Competition        [750]   
         blue green                      Assembly Competition        [750]   
         yellow                          Assembly Competition        [750]   
30048733 gray green blue yellow red      Assembly Competition        [900]   
         gray                            Assembly Competition        [900]   
         blue green                      Assembly Competition        [900]   
         yellow                          Assembly Competition        [900]   
30048670 gray green blue yellow red                      Code        [400]   
         gray                                            Code        [400]   
         blue green                                      Code        [400]   
         yellow                                          Code        [400]   
30048588 gray green blue yellow red                      Code        [900]   
         gray                                            Code        [900]   
         blue green                                      Code        [900]   
         yellow                                          Code        [900]   
30048570 gray green blue yellow red      Assembly Competition       [2700]   
         gray                            Assembly Competition       [2700]   
         blue green                      Assembly Competition       [2700]   
         yellow                          Assembly Competition       [2700]   
30048560 gray green blue yellow red                      Code       [1500]   
         gray                                            Code       [1500]   
         blue green                                      Code       [1500]   
         yellow                                          Code       [1500]   
30048494 gray green blue yellow red                      Code       [1200]   
         gray                                            Code       [1200]   
         blue green                                      Code       [1200]   
         yellow                                          Code       [1200]   
30048537 gray green blue yellow red                      Code        [900]   
         gray                                            Code        [900]   
         blue green                                      Code        [900]   
         yellow                                          Code        [900]   
30048468 gray green blue yellow red  UI Prototype Competition       [2100]   
         gray                        UI Prototype Competition       [2100]   

                                      filter status     price status  \
3

In [ ]:
price_info6.head(40)

In [41]:
chk=price_info6.loc[idx[:, "gray"],["price set"] ].iloc[:,0][1]#.iloc[0][1]
chk


nan

In [ ]:
#filtered_history_ID_old_W_id_i_2
import matplotlib.pyplot as plt
chk=price_info6.loc[idx[30048887, "gray green blue yellow red"],["price set"] ].iloc[0][0]
print(prediction_info6.loc[idx[30048887, "gray green blue yellow red"],["predicted price"] ])
aver=15#(max(chk)-min(chk))/len(chk)
plt.hist(chk, color = 'blue', edgecolor = 'black',    bins = int(aver/1))
# Add labels
plt.title('Histogram of price distribution')
plt.xlabel('price ($)')
plt.ylabel('Frequencey')
